1) Leitura do dataset:

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
import os
from matplotlib import ticker
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
#Modelos de classificação
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier


NOTEBOOK_DIR = os.getcwd()
BASE_DIR = os.path.dirname(NOTEBOOK_DIR)
FIG_DIR = os.path.join(BASE_DIR, "reports", "figures")

nome_real_do_arquivo = 'netflix_titles.csv' 
caminho_do_arquivo = f"../data/raw/{nome_real_do_arquivo}"

df = pd.read_csv(
    caminho_do_arquivo,
    sep=',',           
    encoding='latin-1',
    low_memory=False
)

df.head()



,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


2) Tratando dados duplicados: Valores ausentes em variáveis categóricas foram preenchidos com a categoria ‘Unknown’, evitando perda de informação e viés por remoção de amostras


In [58]:
faltantes_antes = (df.isna().mean() * 100).round(2)
print("Percentual de dados faltantes por coluna antes do tratamento (%):\n")
print(faltantes_antes)

#adicionando unknown nos dados faltantes das colunas abaixo
colunas_categoricas = ["director", "cast", "country"]
df[colunas_categoricas] = df[colunas_categoricas].fillna("Unknown")

#removendo linhas com porcentagem baixa de dados faltantes:
faltantes_depois = (df.isna().mean() * 100).round(2)
print("Percentual de dados faltantes por coluna após o tratamento (%):\n")
print(faltantes_depois)

#obs: Nenhum dado duplicado


Percentual de dados faltantes por coluna antes do tratamento (%):

show_id          0.00
type             0.00
title            0.00
director        29.91
cast             9.37
country          9.44
date_added       0.11
release_year     0.00
rating           0.05
duration         0.03
listed_in        0.00
description      0.00
dtype: float64
Percentual de dados faltantes por coluna após o tratamento (%):

show_id         0.00
type            0.00
title           0.00
director        0.00
cast            0.00
country         0.00
date_added      0.11
release_year    0.00
rating          0.05
duration        0.03
listed_in       0.00
description     0.00
dtype: float64


3. Transformação de variáveis

In [59]:

#conversão 
df["date_added"] = pd.to_datetime(df["date_added"], errors="coerce")
df["year_added"] = df["date_added"].dt.year
df["month_added"] = df["date_added"].dt.month

df.dropna(subset=["year_added", "month_added"], inplace=True)

df["year_added"] = df["year_added"].astype(int)
df["month_added"] = df["month_added"].astype(int)




4.Tratar duration (converter para valor numérico)


In [60]:
# Extrai apenas os números da coluna duration, usando Int64 para aceitar NaN
df["duration_num"] = df["duration"].str.extract("(\d+)").astype("Int64")

# Extrai o tipo: min ou Season
df["duration_type"] = df["duration"].str.extract("(min|Season)")

df[["duration", "duration_num", "duration_type"]].head(10)


,duration,duration_num,duration_type
0,90 min,90,min
1,2 Seasons,2,Season
2,1 Season,1,Season
3,1 Season,1,Season
4,2 Seasons,2,Season
5,1 Season,1,Season
6,91 min,91,min
7,125 min,125,min
8,9 Seasons,9,Season
9,104 min,104,min


5.Padronizar colunas textuais (title, director, cast, country)

In [61]:
# Lista de colunas textuais
colunas_texto = ["title", "director", "cast", "country"]

for col in colunas_texto:
    df[col] = df[col].str.strip()

df[colunas_texto].head(10)

,title,director,cast,country
0,Dick Johnson Is Dead,Kirsten Johnson,Unknown,United States
1,Blood & Water,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa
2,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Unknown
3,Jailbirds New Orleans,Unknown,Unknown,Unknown
4,Kota Factory,Unknown,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India
5,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",Unknown
6,My Little Pony: A New Generation,"Robert Cullen, JosÃ© Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",Unknown
7,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin..."
8,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom
9,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States


6. Separar múltiplos valores em listed_in (gêneros)

In [62]:
# Transformar cada string de gêneros em lista
df["genres_list"] = df["listed_in"].str.split(", ")
df[["listed_in", "genres_list"]].head(10)


,listed_in,genres_list
0,Documentaries,[Documentaries]
1,"International TV Shows, TV Dramas, TV Mysteries","[International TV Shows, TV Dramas, TV Mysteries]"
2,"Crime TV Shows, International TV Shows, TV Act...","[Crime TV Shows, International TV Shows, TV Ac..."
3,"Docuseries, Reality TV","[Docuseries, Reality TV]"
4,"International TV Shows, Romantic TV Shows, TV ...","[International TV Shows, Romantic TV Shows, TV..."
5,"TV Dramas, TV Horror, TV Mysteries","[TV Dramas, TV Horror, TV Mysteries]"
6,Children & Family Movies,[Children & Family Movies]
7,"Dramas, Independent Movies, International Movies","[Dramas, Independent Movies, International Mov..."
8,"British TV Shows, Reality TV","[British TV Shows, Reality TV]"
9,"Comedies, Dramas","[Comedies, Dramas]"


7. Selecionar variáveis relevantes

In [63]:
df_model = df[[
    "type",
    "director",
    "cast",
    "country",
    "year_added",
    "month_added",
    "release_year",
    "rating",
    "duration_num",
    "duration_type",
    "genres_list",
    "description"
]]

df_model.head()

,type,director,cast,country,year_added,month_added,release_year,rating,duration_num,duration_type,genres_list,description
0,Movie,Kirsten Johnson,Unknown,United States,2021,9,2020,PG-13,90,min,[Documentaries],"As her father nears the end of his life, filmm..."
1,TV Show,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021,9,2021,TV-MA,2,Season,"[International TV Shows, TV Dramas, TV Mysteries]","After crossing paths at a party, a Cape Town t..."
2,TV Show,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Unknown,2021,9,2021,TV-MA,1,Season,"[Crime TV Shows, International TV Shows, TV Ac...",To protect his family from a powerful drug lor...
3,TV Show,Unknown,Unknown,Unknown,2021,9,2021,TV-MA,1,Season,"[Docuseries, Reality TV]","Feuds, flirtations and toilet talk go down amo..."
4,TV Show,Unknown,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021,9,2021,TV-MA,2,Season,"[International TV Shows, Romantic TV Shows, TV...",In a city of coaching centers known to train I...


8. Codificar variáveis categóricas:

In [64]:
df_encoded = pd.get_dummies(
    df_model,
    columns=["type", "rating", "duration_type"],
    drop_first=True 
)

df_encoded.head()


mlb = MultiLabelBinarizer()

genres_encoded = pd.DataFrame(
    mlb.fit_transform(df_model["genres_list"]),
    columns=mlb.classes_,
    index=df_model.index
)

# Concatenar ao dataframe principal
df_encoded = pd.concat([df_encoded.drop("genres_list", axis=1), genres_encoded], axis=1)

top_countries = df_model["country"].value_counts().nlargest(10).index
df_encoded["country_top"] = df_model["country"].apply(lambda x: x if x in top_countries else "Other")
df_encoded = pd.get_dummies(df_encoded, columns=["country_top"], drop_first=True)

df_encoded.head()



,director,cast,country,year_added,month_added,release_year,duration_num,description,type_TV Show,rating_74 min,...,country_top_France,country_top_India,country_top_Japan,country_top_Mexico,country_top_Other,country_top_South Korea,country_top_Spain,country_top_United Kingdom,country_top_United States,country_top_Unknown
0,Kirsten Johnson,Unknown,United States,2021,9,2020,90,"As her father nears the end of his life, filmm...",False,False,...,False,False,False,False,False,False,False,False,True,False
1,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021,9,2021,2,"After crossing paths at a party, a Cape Town t...",True,False,...,False,False,False,False,True,False,False,False,False,False
2,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Unknown,2021,9,2021,1,To protect his family from a powerful drug lor...,True,False,...,False,False,False,False,False,False,False,False,False,True
3,Unknown,Unknown,Unknown,2021,9,2021,1,"Feuds, flirtations and toilet talk go down amo...",True,False,...,False,False,False,False,False,False,False,False,False,True
4,Unknown,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021,9,2021,2,In a city of coaching centers known to train I...,True,False,...,False,True,False,False,False,False,False,False,False,False


9. Problema de classificação: Movie ou TV Show?

In [65]:

# Separar features e target
X = df_encoded.drop(["type_TV Show", "type_Movie", "type_TV Show"], axis=1, errors='ignore')
y = df_encoded["type_TV Show"]
# Dividir treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train = pd.get_dummies(X_train, columns=["director", "cast", "country"], drop_first=True)
X_test  = pd.get_dummies(X_test, columns=["director", "cast", "country"], drop_first=True)

num_cols = X_train.select_dtypes(include=["int64", "float64", "Int64"]).columns

imputer = SimpleImputer(strategy="constant", fill_value=0)

X_train[num_cols] = imputer.fit_transform(X_train[num_cols])
X_test[num_cols]  = imputer.transform(X_test[num_cols])

X_numeric = X.select_dtypes(include=['number'])

imputer = SimpleImputer(strategy="constant", fill_value=0)
X_numeric_imputed = pd.DataFrame(imputer.fit_transform(X_numeric), columns=X_numeric.columns)

#Definir modelos
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "DecisionTree": DecisionTreeClassifier(),
    "RandomForest": RandomForestClassifier(),
    "GradientBoosting": GradientBoostingClassifier(),
    "SVM": SVC()
}

#Validar cada modelo com cross-validation
for name, model in models.items():
    scores = cross_val_score(model, X_numeric_imputed, y, cv=5, scoring='f1')  # F1-score médio
    print(f"{name} F1-score médio: {scores.mean():.4f}")

LogisticRegression F1-score médio: 0.9983
DecisionTree F1-score médio: 0.9986
RandomForest F1-score médio: 0.9988
GradientBoosting F1-score médio: 0.9981
SVM F1-score médio: 0.9596


10. Analise de desbalanceamento.

In [66]:
# Garantir apenas colunas numéricas
X_train_num = X_train.select_dtypes(include=[np.number])
X_test_num  = X_test.select_dtypes(include=[np.number])

# Imputação
imputer = SimpleImputer(strategy="constant", fill_value=0)
X_train_imputed = imputer.fit_transform(X_train_num)
X_test_imputed  = imputer.transform(X_test_num)

# Treinar modelo
model = RandomForestClassifier(random_state=42)
model.fit(X_train_imputed, y_train)

# Predição
y_pred = model.predict(X_test_imputed)

# Avaliação
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

       False       1.00      1.00      1.00      1226
        True       1.00      1.00      1.00       516

    accuracy                           1.00      1742
   macro avg       1.00      1.00      1.00      1742
weighted avg       1.00      1.00      1.00      1742

[[1226    0]
 [   1  515]]


11. Preparação final dos dados para os modelos de classificação

In [67]:
X_train_clean = pd.DataFrame(X_train_imputed, columns=X_train_num.columns)
X_test_clean  = pd.DataFrame(X_test_imputed, columns=X_test_num.columns)

# Verificação final
non_numeric_cols = X_train_clean.select_dtypes(exclude=[np.number]).columns
print("Colunas não numéricas restantes:", non_numeric_cols)


Colunas não numéricas restantes: Index([], dtype='object')


12. Ajuste de hiperparâmetros do Random Forest com GridSearchCV


In [68]:
# Definição de um grid reduzido de hiperparâmetros
# O grid foi mantido pequeno para diminuir o tempo de execução, mas ainda permitir avaliar combinações relevantes.

param_grid_small = {
    "n_estimators": [50, 100],                 # menos árvores para rodar rápido
    "max_depth": [None, 10],                   # profundidade total ou limitada
    "min_samples_split": [2, 5],               # dividir nó com poucas amostras
    "min_samples_leaf": [1, 2],                # folhas pequenas
    "max_features": ["sqrt"]                   # padrão sqrt para Random Forest
}

# Criar modelo base
rf_small = RandomForestClassifier(random_state=42)

# Configurar GridSearchCV rápido
grid_search_small = GridSearchCV(
    estimator=rf_small,
    param_grid=param_grid_small,
    cv=3,                            # validação cruzada 3-fold para rodar mais rápido
    scoring="f1",
    n_jobs=-1,
    verbose=1
)

# Treinar
grid_search_small.fit(X_train_clean, y_train)

print("Melhores hiperparâmetros encontrados:", grid_search_small.best_params_)
print("Melhor F1-score médio:", grid_search_small.best_score_)


Fitting 3 folds for each of 16 candidates, totalling 48 fits
Melhores hiperparâmetros encontrados: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Melhor F1-score médio: 0.9983061969477064


13. Ajuste de hiperparâmetros do modelo Decision Tree com GridSearchCV


In [69]:
# Grid simples de hiperparâmetros para Decision Tree
param_grid_dt = {
    "max_depth": [None, 5, 10],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5]
}

# Modelo base
dt = DecisionTreeClassifier(random_state=42)

# GridSearch com validação cruzada
grid_search_dt = GridSearchCV(
    estimator=dt,
    param_grid=param_grid_dt,
    cv=3,              # validação cruzada
    scoring="f1",      # métrica adequada para desbalanceamento
    n_jobs=-1,
    verbose=1
)

# Treinamento
grid_search_dt.fit(X_train_clean, y_train)

# Resultados
print("Melhores hiperparâmetros encontrados:")
print(grid_search_dt.best_params_)

print("Melhor F1-score médio:")
print(grid_search_dt.best_score_)


Fitting 3 folds for each of 27 candidates, totalling 81 fits


Melhores hiperparâmetros encontrados:
{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Melhor F1-score médio:
0.9983047908276667


14. Treinamento do modelo final Decision Tree com hiperparâmetros otimizados


In [70]:
# Recuperar o melhor modelo Decision Tree
best_dt = grid_search_dt.best_estimator_

# Treinar com os dados de treino
best_dt.fit(X_train_clean, y_train)


,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",42
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current

15. Avaliação do modelo Decision Tree no conjunto de teste

In [71]:
# Predições
y_pred_dt = best_dt.predict(X_test_clean)

print("Decision Tree - Métricas no conjunto de teste")
print(classification_report(y_test, y_pred_dt))


Decision Tree - Métricas no conjunto de teste
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      1226
        True       1.00      1.00      1.00       516

    accuracy                           1.00      1742
   macro avg       1.00      1.00      1.00      1742
weighted avg       1.00      1.00      1.00      1742



16. Treinamento do modelo final Random Forest com hiperparâmetros otimizados

In [72]:
# Recuperar o melhor Random Forest do GridSearch
best_rf = grid_search_small.best_estimator_

# Treinar novamente com os dados de treino
best_rf.fit(X_train_clean, y_train)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",50
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric(

17. Comparação final de desempenho entre os modelos de classificação

In [74]:
models_final = {
    "Random Forest (Final)": best_rf,    # Modelo Random Forest otimizado
    "Decision Tree (Ajustada)": best_dt  # Modelo Decision Tree otimizado
}

results = []

for name, model in models_final.items():
    y_pred = model.predict(X_test_clean)

  # Calcular métricas de desempenho

    results.append({
        "Modelo": name,
        "Accuracy": accuracy_score(y_test, y_pred),       # Acurácia geral
        "Precision": precision_score(y_test, y_pred),     # Precisão da classe positiva
        "Recall": recall_score(y_test, y_pred),           # Recall da classe positiva
        "F1-score": f1_score(y_test, y_pred)              # F1-score (equilíbrio entre precisão e recall)
    })

df_results_final = pd.DataFrame(results)
df_results_final


,Modelo,Accuracy,Precision,Recall,F1-score
0,Random Forest (Final),0.999426,1.0,0.998062,0.99903
1,Decision Tree (Ajustada),0.999426,1.0,0.998062,0.99903


##### CONCLUSÃO DA ETAPA DE CLASSIFICAÇÃO:
Embora *Random Forest* e *Decision Tree* tenham apresentado métricas idênticas no conjunto de teste, o **Random Forest** foi escolhido como modelo final por apresentar maior robustez e melhor capacidade de generalização, conforme observado na validação cruzada, além de menor risco de overfitting.

